In [16]:
from tqdm.auto import tqdm
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn import model_selection, naive_bayes, svm
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report

In [2]:
df1 = pd.read_csv("/Users/julianalexander/Library/Mobile Documents/com~apple~CloudDocs/HSG/1. Semester MBI FS2021/8,049,1.00 Text Mining mit Python/TextMining21/Reddit_Julian_TOTAL-klassifiziert.csv")
df2 = pd.read_csv("/Users/julianalexander/Library/Mobile Documents/com~apple~CloudDocs/HSG/1. Semester MBI FS2021/8,049,1.00 Text Mining mit Python/TextMining21/Reddit_Marek_klassifiziert.csv")
df3 = pd.read_csv("/Users/julianalexander/Library/Mobile Documents/com~apple~CloudDocs/HSG/1. Semester MBI FS2021/8,049,1.00 Text Mining mit Python/TextMining21/Reddit_Yonas_Total_Klassifiziert.csv")

In [3]:
Atweets = pd.concat([df1, df2, df3], axis=0, join='inner')
Atweets = Atweets.sort_values(by='index')
Atweets = Atweets.reset_index(drop=True)

Punktuierung

In [ ]:
def remove_punctuation(txt):
    txt_nopunt = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunt

In [ ]:
Atweets["submission"] = Atweets["submission"].apply(lambda x: remove_punctuation(x))

Tokenisierung

In [ ]:
def tokenize(txt):
    tokens = re.split("\W+", txt)
    return tokens

In [ ]:
Atweets["submissions_tokenized"] = Atweets["submission"].apply(lambda x: tokenize(x.lower()))

Stopwörter

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")

In [ ]:
stopwords.append("ll")
stopwords.append("im")
stopwords.append("youre")
stopwords.append("youve")
stopwords.append("youll")
stopwords.append("youd")
stopwords.append("shes")
stopwords.append("its")
stopwords.append("thatll")
stopwords.append("dont")
stopwords.append("shouldve")
stopwords.append("arent")
stopwords.append("couldnt")
stopwords.append("didnt")
stopwords.append("doesnt")
stopwords.append("hadnt")
stopwords.append("havent")
stopwords.append("isnt")
stopwords.append("mightnt")
stopwords.append("neednt")
stopwords.append("shant")
stopwords.append("shouldnt")
stopwords.append("wasnt")
stopwords.append("werent")
stopwords.append("wont")
stopwords.append("wouldnt")

In [ ]:
def remove_stopwords(txt_tokenized):
    txt_clean = [word for word in txt_tokenized if word not in stopwords]
    return txt_clean

In [ ]:
Atweets["submission_no_stopwords"] = Atweets["submissions_tokenized"].apply(lambda x: remove_stopwords(x))

Text-Bereinigung

In [ ]:
def clean_text(text):
    text = "".join([c for c in text if c not in string.punctuation])
    tokens = re.split("\W+", text)
    text = [word for word in tokens if word not in stopwords]
    return text

In [ ]:
Atweets["submission_nostop"] = Atweets["submission"].apply(lambda x: clean_text(x.lower()))

Stemming

In [ ]:
ps = PorterStemmer()

In [ ]:
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text

In [ ]:
Atweets["submission_stemmed"] = Atweets["submission_nostop"].apply(lambda x: stemming(x))

Lemmatisierung

In [ ]:
wn = nltk.WordNetLemmatizer()

In [ ]:
def lemmatization(token_txt):
    text = [wn.lemmatize(word) for word in token_txt]
    return text

In [ ]:
Atweets["submission_lemmatized"] = Atweets["submission_nostop"].apply(lambda x: lemmatization(x))

In [ ]:
Atweets.shape

In [ ]:
Atweets["submission"]

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Atweets["submission_stemmed"],Atweets['sentiment'],test_size=0.2)




In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=12842)
Tfidf_vect.fit(Atweets["submission_stemmed"])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

TEST

In [4]:
# Step - a : Remove blank rows if any.
Atweets['submission'].dropna(inplace=True)
# Step - b : Change all the text to lower case. This is required as python interprets 'dog' and 'DOG' differently
Atweets['submission'] = [entry.lower() for entry in Atweets['submission']]
# Step - c : Tokenization : In this each entry in the corpus will be broken into set of words
Atweets['submission']= [word_tokenize(entry) for entry in Atweets['submission']]
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV
for index,entry in enumerate(Atweets['submission']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Atweets.loc[index,'text_final'] = str(Final_words)

In [5]:
Atweets.isnull().sum().sum()

0

In [6]:
df1 = Atweets[Atweets.isna().any(axis=1)]

In [7]:
df1

,index,submission,sentiment,text_final


In [ ]:
Atweets['sentiment'].astype(int)

In [8]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Atweets["text_final"],Atweets['sentiment'],test_size=0.2)


In [9]:
Tfidf_vect = TfidfVectorizer(max_features=12842)
Tfidf_vect.fit(Atweets["text_final"])
Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [ ]:
Atweets = Atweets.reset_index()

In [ ]:
print(Train_X_Tfidf)

In [ ]:
print(Train_Y)

In [18]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)

Naive Bayes Accuracy Score ->  75.04159733777038


In [20]:
precision_recall_fscore_support(predictions_NB, Test_Y, average='weighted')




/Users/julianalexander/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.9961273982889874, 0.7504159733777038, 0.8559878023319815, None)

In [22]:
print(predictions_NB[:10])

[3 3 3 3 3 3 3 3 3 3]


In [21]:
print(Test_Y[:10])

1766    3
2684    3
1624    3
2864    3
1291    3
487     3
2354    3
2858    4
344     4
1454    4
Name: sentiment, dtype: int64


In [12]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

SVM Accuracy Score ->  75.04159733777038
